In [46]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [47]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [48]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [49]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [50]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2832) (12128, 2832)


In [51]:
features_list =[
           
'distinct_merchant',
'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'ENTROPY(raw.mode_cleaned)#720',
'max_one_time_spent',
'amt_first_android_txn',
'MAX(pg_success_raw_data.amount)',
'top_payment_mode_count',
'amt_first_cc_txn',
'STD(pg_success_raw_data.amount)',
'month_year',
'90_in_24mob' ,
    'Weight_vintage'
 ]




In [52]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3160419/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3160419/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [54]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 12), (15604, 12), (23406,), (15604,))

In [57]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 12) (4682, 12) (15604, 12)


In [58]:
#Get vintage weight values
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [59]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [60]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [61]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 1,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7,
    'alpha':5,
    'gamma':5,
   
     'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 100,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.55745	eval-auc:0.53978
[10]	train-auc:0.57491	eval-auc:0.56098
[20]	train-auc:0.57910	eval-auc:0.56629
[30]	train-auc:0.58010	eval-auc:0.56666
[40]	train-auc:0.58069	eval-auc:0.56838
[50]	train-auc:0.58102	eval-auc:0.56964
[60]	train-auc:0.58198	eval-auc:0.56957
[70]	train-auc:0.58217	eval-auc:0.56968
[80]	train-auc:0.58212	eval-auc:0.56961
[90]	train-auc:0.58217	eval-auc:0.57045
[99]	train-auc:0.58229	eval-auc:0.57033


In [62]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.58, Gini: 0.16, KS: 0.12
Eval AUC: 0.57, Gini: 0.14, KS: 0.12
Test AUC: 0.54, Gini: 0.09, KS: 0.07
OOT AUC: 0.56, Gini: 0.12, KS: 0.10


In [63]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.217198         0.207215        0.208763        0.197419   5617   
7          0.182451         0.186589        0.197412        0.180002   1869   
6          0.163996         0.177538        0.179999        0.175434   1872   
5          0.155286         0.173955        0.175416        0.172773   1816   
4          0.152850         0.170757        0.172769        0.167103   1930   
3          0.157417         0.164301        0.167097        0.162379   1874   
2          0.123296         0.160937        0.162375        0.159658   1687   
1          0.098106         0.159336        0.159631        0.159259   2059   

        label_good  label_bad  
Decile                         
8             4397       1220  
7             1528        341  
6             1565        307  
5             1534        282  
4             1635        295  
3             1579        295  
2             1479        208  
1             1857        202

In [64]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.199316         0.207255        0.208763        0.196733   4681   
7          0.178846         0.186347        0.196726        0.179909   1560   
6          0.168590         0.177378        0.179908        0.175234   1560   
5          0.168456         0.173868        0.175231        0.172773   1490   
4          0.167892         0.170778        0.172769        0.167103   1632   
3          0.152021         0.164332        0.167097        0.162468   1559   
2          0.133705         0.160953        0.162463        0.159697   1436   
1          0.141163         0.159330        0.159631        0.159259   1686   

        label_good  label_bad  
Decile                         
8             3748        933  
7             1281        279  
6             1297        263  
5             1239        251  
4             1358        274  
3             1322        237  
2             1244        192  
1             1448        238

In [69]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.208184         0.207443        0.208763        0.200227   3617   
7          0.186537         0.189491        0.200190        0.181218   1233   
6          0.172158         0.178578        0.181213        0.176475   1214   
5          0.187864         0.174958        0.176473        0.173320   1203   
4          0.163800         0.172101        0.173313        0.170410   1221   
3          0.145775         0.166405        0.170405        0.163661   1077   
2          0.124814         0.161961        0.163650        0.160500   1346   
1          0.119967         0.159405        0.160459        0.159259   1217   

        label_good  label_bad  
Decile                         
8             2864        753  
7             1003        230  
6             1005        209  
5              977        226  
4             1021        200  
3              920        157  
2             1178        168  
1             1071        146

In [67]:
#Save the model
xgb_model.save_model('LeoPayu_PG_weightVintage_90_in_24mob.json')
print('Model Saved')

Model Saved


In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
6                    MAX(pg_success_raw_data.amount)        23.0
2  MEAN(raw.amount WHERE device_os_category = Win...        20.0
4                                 max_one_time_spent        18.0
9                    STD(pg_success_raw_data.amount)        13.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         8.0
7                             top_payment_mode_count         5.0
5                              amt_first_android_txn         5.0
8                                   amt_first_cc_txn         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         3.0
0                                  distinct_merchant         1.0


In [17]:
feature_importance_df

Feature  Importance
6                    MAX(pg_success_raw_data.amount)        23.0
2  MEAN(raw.amount WHERE device_os_category = Win...        20.0
4                                 max_one_time_spent        18.0
9                    STD(pg_success_raw_data.amount)        13.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         8.0
7                             top_payment_mode_count         5.0
5                              amt_first_android_txn         5.0
8                                   amt_first_cc_txn         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         3.0
0                                  distinct_merchant         1.0

In [68]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_PG_Train_Pred_9024_Vin.csv',index=False)
Eval.to_csv('LeoPayu_PG_Eval_Pred_9024_Vin.csv',index=False)
Test.to_csv('LeoPayu_PG_Test_Pred_9024_Vin.csv',index=False)
Oot.to_csv('LeoPayu_PG_Oot_Pred_9024_Vin.csv',index=False)


In [19]:
#Save the model
xgb_model.save_model('LeoPayu_PG_90_in_24mob.json')
print('Model Saved')

Model Saved
